In [5]:
from typing import List
import requests
import json
import ollama
import logging
import pandas as pd
import re
from typing import Optional


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LlamaQASystem:
    def __init__(self, model_name: str = "llama3.2-vision"):
        self.model_name = model_name

    def determine_question_type(self,query: str) -> str:
        """
        回傳: "multiple_choice" | "true_false" | "qa"
        支援中英文 & 各種標點
        """
        q = query.strip().lower()

        # --- Multiple‑choice --------------------------------------------------
        # 1) 行首或換行後出現  A～D / 全形Ａ～Ｄ / 「答」，
        #    後面接　. ． : ： 、)
        mc_pattern = re.compile(r'(?:^|\n)\s*(?:[a-dａ-ｄ]|答)[:\.．:：、\)]', re.I)
        # 2) or 句子帶 "which of the following"
        mc_keywords_en = ["which of the following", "which one of the following",
                        "which option", "choose one of"]

        if mc_pattern.search(query) or any(kw in q for kw in mc_keywords_en):
            return "multiple_choice"

        # --- True / False -----------------------------------------------------
        tf_keywords_zh = ["是否", "是嗎", "對嗎", "正確嗎"]
        tf_keywords_en = ['true or false', 'is it', 'is this', 'is that', 
             'is it possible', 'correct or not']

        if any(k in q for k in tf_keywords_zh + tf_keywords_en):
            return "true_false"

        # --- Default ----------------------------------------------------------
        return "qa"


    def display_response(self, query: str,question_type: Optional[str] = None):
            """Display response with text and images"""
            try:

                response_text = self.generate_response(query,question_type)
                
                print("Question:", query)
                print("\nSystem Response:")
                print(response_text)
                print("\n" + "-"*50 + "\n")

                return response_text 
                    
            except Exception as e:
                logger.error(f"Error in display_response: {str(e)}", exc_info=True)  
                return "", [] 

    def generate_response(self, query: str,question_type: Optional[str] = None) -> str:
        # --- ① 題型 --------------------------------------------------------
        q_type = question_type or self.determine_question_type(query)
        user_prompt = f"""
            【問題】
            {query}\n
            """
        if q_type == "multiple_choice":
                format_rules = (
                    "這是一題選擇題，回答格式如下：\n"
                    "先根據題目整理參考資訊、你的理解與常識\n"
                    "用 2-3 句話說明理由。\n"
                    "最後再給出答案，只能回答 A/B/C/D (請勿帶任何標點、文字、也不要只回答選項內容)\n"
                    "若同時出現多個選項，請只選一個最適合的\n"
                )
        elif q_type == "true_false":
                format_rules = (
                    "這是一題是非題，請按照下列格式回答：\n"
                    "先根據題目整理參考資訊、你的理解與常識\n"
                    "最後再給出答案，只能寫「True」或「False」\n"
                )
        else:   # qa
                format_rules = (
                    "請依以下格式回答：\n"
                    "針對問題提供具體答案並詳細說明 \n"
                )

            #"您是一名專業獸醫，1.擅長犬認知功能障礙綜合症（CCD）的診斷和護理 2.擁有豐富的寵物中醫知識 3.常見問題診斷及改善建議\n" "請在答案最後顯示你參考的來源連結或論文名稱，如果來源中包含「(經驗) some_link」，請在回答中以 [Experience: some_link] 形式標示；若包含「(文獻) some.pdf」，就 [reference: some.pdf]\n""如檢索結果仍無相關資訊，請以[NoRef]標示並根據你的常識回答。\n"
        system_prompt = (
                """你是資深獸醫，擅長犬認知功能障礙綜合症（CCD）的診斷和護理並擁有豐富的寵物中醫知識，必須遵守以下規則回答問題：
                    1. 若需補充一般臨床常識，請將該句放在段落最後並標註［常識］。
                    2. 每一句結尾必須標註引用來源編號，如［1］或［1,3］。
                    3. 並在最後面整理列出每個編號的source_file，如[1] ...pdf 或 [2] Chinese Veterinary Materia Medica """
                + format_rules
            )
        message = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_prompt},
            ]

        response = ollama.chat(
            model=self.model_name,
            messages=message
        )
        return response['message']['content']

In [2]:
import re

def parse_llm_answer(resp: str, q_type: str) -> str:
    """
    解析 LLM 回答文字，回傳最終答案：
      • multiple_choice → 'A'|'B'|'C'|'D'|'UNK'
      • true_false      → 'TRUE'|'FALSE'|'UNK'
    """
    txt = resp.lower()
    txt = re.sub(r'[，。、．；：\s]+', ' ', txt)        # 先統一空白

    if q_type == "multiple_choice":
        # 找所有「獨立」的 a-d (含大小寫)，不含 '選項a' 這種組字
        matches = re.findall(r'\b([abcd])\b', txt, flags=re.I)
        return matches[-1].upper() if matches else "UNK"

    elif q_type == "true_false":
        # 找所有 true/false / 對/錯 / 是/否
        tf_matches = re.findall(
            r'\b(true|false|正確|錯誤|對|錯|是|否)\b', txt)
        if not tf_matches:
            return "UNK"
        last = tf_matches[-1]
        return "TRUE" if last in ("true", "正確", "對", "是") else "FALSE"

    else:   # 其餘題型原文返回
        return resp


測試

In [6]:
# 建立 QA 系統，沿用同一個 embedding_processor
qa_system = LlamaQASystem(model_name='llama3.2-vision')

In [7]:
qa_system.display_response("the benefits of MCT Oil")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: the benefits of MCT Oil

System Response:
MCT油（Medium Chain Triglyceride oil）是一種來自椰子、乳脂和豬脂的精煉油脂，主要含有中鏈三酯。它已被證明具有多項益處，包括：

1. 提高能量：MCT油中的中鏈脂肪可以快速地吸收和利用為能量，從而提高體內的能量水平。
2. 改善營養狀態：MCT油富含維生素、微量元素和抗氧化劑，有助於改善整體健康狀態。
3. 支持神經系統功能：中鏈脂肪可以幫助支持大腦和神經系統的功能，從而提高思維清晰度和記憶力。
4. 促進新陳代謝：MCT油中的中鏈脂肪可以幫助加速新陳代謝率，促進體內廢物排泄。
5. 支持免疫系統功能：MCT油富含抗氧化劑和免疫活性成分，有助於支持免疫系統的功能。

　　在犬類中，MCT油也具有以下益處：

1. 改善肠胃健康：MCT油可以幫助改善犬類的肠胃健康狀態，減少便秘和腸炎等問題。
2. 支持皮膚健康：MCT油中的抗氧化劑和維生素有助於支持犬類的皮膚健康。

　　總之，MCT油是一種健康益處多多的補充品，可以幫助改善整體健康狀態和提高能量水平。

［常識］MCT油通常以液體或凝膠形式提供，使用時建議與食物混合並嚴格按照製造商的指示進行服用。


　　參考來源：
1. "Medium-Chain Triglycerides" by National Institutes of Health (NIH) [1]
2. "MCT Oil Benefits and Side Effects" by Healthline [2]

　　相關文獻：
[1] National Institutes of Health. (2020). Medium-Chain Triglycerides.
https://ods.od.nih.gov/factsheets/MCT/
[2] Healthline. (2022). MCT Oil Benefits and Side Effects.

　　參考來源文獻：
MCT油的相關文獻包括研究論文和醫學評論文章，以下是部分典型的文獻：

1. St-Onge et al. (2014) "Medium-Chain Triglycerides Increase Energy Expenditure an

'MCT油（Medium Chain Triglyceride oil）是一種來自椰子、乳脂和豬脂的精煉油脂，主要含有中鏈三酯。它已被證明具有多項益處，包括：\n\n1. 提高能量：MCT油中的中鏈脂肪可以快速地吸收和利用為能量，從而提高體內的能量水平。\n2. 改善營養狀態：MCT油富含維生素、微量元素和抗氧化劑，有助於改善整體健康狀態。\n3. 支持神經系統功能：中鏈脂肪可以幫助支持大腦和神經系統的功能，從而提高思維清晰度和記憶力。\n4. 促進新陳代謝：MCT油中的中鏈脂肪可以幫助加速新陳代謝率，促進體內廢物排泄。\n5. 支持免疫系統功能：MCT油富含抗氧化劑和免疫活性成分，有助於支持免疫系統的功能。\n\n\u3000\u3000在犬類中，MCT油也具有以下益處：\n\n1. 改善肠胃健康：MCT油可以幫助改善犬類的肠胃健康狀態，減少便秘和腸炎等問題。\n2. 支持皮膚健康：MCT油中的抗氧化劑和維生素有助於支持犬類的皮膚健康。\n\n\u3000\u3000總之，MCT油是一種健康益處多多的補充品，可以幫助改善整體健康狀態和提高能量水平。\n\n［常識］MCT油通常以液體或凝膠形式提供，使用時建議與食物混合並嚴格按照製造商的指示進行服用。\n\n\n\u3000\u3000參考來源：\n1. "Medium-Chain Triglycerides" by National Institutes of Health (NIH) [1]\n2. "MCT Oil Benefits and Side Effects" by Healthline [2]\n\n\u3000\u3000相關文獻：\n[1] National Institutes of Health. (2020). Medium-Chain Triglycerides.\nhttps://ods.od.nih.gov/factsheets/MCT/\n[2] Healthline. (2022). MCT Oil Benefits and Side Effects.\n\n\u3000\u3000參考來源文獻：\nMCT油的相關文獻包括研究論文和醫學評論文章，以下是部分典型的文獻：\n\n1. St-Onge et al. (2014) "Medium-Chain Triglycerides I

In [12]:
# 1. 讀檔 + 題型篩選
df = pd.read_excel("test_questions_en.xlsx")#test_questions #test_questions_withANS
test_df = df[df["type"].isin(["multiple_choice", "true_false"])].copy()
# test_df = df[df["type"].isin(["qa"])].copy()

# 2. ★ 建立欄位（一定要在後面的篩選前先加）
test_df["llm_response"] = ""
test_df["predicted"]    = ""
test_df["is_correct"]   = 0

# 3. 再依 domain 篩子集合
# test_df = test_df[test_df["domain"] == "中醫"].copy()
# test_df=test_df.head(5)

In [13]:
dataset = [] # for ragas

# 4. 迴圈計分
for idx, row in test_df.iterrows():
    q  = row["question_en"]
    q_type = row["type"]
    gt = str(row["answers"]).strip()
    # ref_ctx   = [ str(row["RAG"]) ] 

    resp= qa_system.display_response(q, q_type)

    if not resp.strip():
        print(f"[WARN] id={row['id']}  LLM 回傳空白")


    resp = qa_system.generate_response(q, q_type)
    pred = parse_llm_answer(resp, q_type)

    test_df.at[idx, "llm_response"] = resp
    test_df.at[idx, "predicted"]    = pred
    test_df.at[idx, "is_correct"]   = int(pred.upper() == gt.upper())


    dataset.append({
        "user_input":           str(q),           # question
        "response":             str(resp),        # llm response
        "reference":            gt                # answers
    })


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: According to TCVM concepts, "Deficiency Pattern" is typically seen in young and healthy animals

System Response:
根據中醫理論，「虛證型」（Deficiency Pattern）通常見於年幼和健康的動物。這種情況下，動物的體質尚未成熟，或者身體功能尚未完全發育，因此易出現虛證型的表現，如疲勞、食欲不振等。

［常識］動物的中醫診斷需要根據個體差異進行調整，並考慮到動物的年齡、性別、品種和身體狀況等多方面因素。［1］

答案：False

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: In the TCVM "Eight Principles" method, which pair of concepts describes the depth of a disease, such as "Exterior" and "Interior"?
A) Cold and Heat
B) Deficiency and Excess
C) Exterior and Interior
D) Yin and Yang

System Response:
根據TCVM的八則原理（Eight Principles）方法，我們知道這些概念用於描述疾病的深度和範圍。其中，“Exterior”和“Interior”是指疾病的範圍，描述疾病是否影響身體內外部組織。

因此，我們可以結合TCVM的八則原理方法和常識得出答案。

根據TCVM的八則原理方法，描述疾病深度的概念是 Exterior 和 Interior。［1］

所以，最適合的選項是：

C) Exterior and Interior


[1] 《Traditional Chinese Veterinary Medicine》

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: CCD only appears in dogs over the age of 14.

System Response:
犬認知功能障礙綜合症（CCD）通常發生在中老年犬，尤其是8歲以上的犬。然而，並非所有14歲以上的犬都會患有CCD。

答案：False

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: If a dog gets stuck in the furniture, it must be a loss of vision and has nothing to do with CCD.

System Response:
這個說法是錯誤的。犬認知功能障礙綜合症（CCD）是一種神經退行性疾病，可能導致一系列的認知功能問題，包括視覺、空間定位和運動協調等方面的問題。當狗被困在家具中時，可能與視覺問題有關，但也可能與 CCD 的其他症狀相關，例如空間定位障礙。

因此，答案是：False

［常識］
犬認知功能障礙綜合症（CCD）是一種神經退行性疾病，影響一系列的認知功能，包括視覺、空間定位和運動協調等。 [1]

參考資訊：
[1] Dodick, D. W., & Weaver, R. (2014). Canine cognitive dysfunction: a review of the literature. Journal of Veterinary Behavior, 9(3), e5-e13. doi: 10.1016/j.jveb.2014.02.005

Source_file：
[1] .pdf

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [14]:
# 5. 計算 Accuracy
overall_acc = test_df["is_correct"].mean()

print("\n=== 每個 domain 的 Accuracy ===")
domain_stats = (
    test_df.groupby("domain")["is_correct"]
           .agg(["count", "sum"])
           .reset_index()
           .rename(columns={"sum": "correct"})
)
domain_stats["accuracy"] = domain_stats["correct"] / domain_stats["count"]
print(domain_stats.to_string(index=False, 
      formatters={"accuracy": "{:.2%}".format}))

print(f"\nOVERALL Accuracy = {overall_acc:.2%}")


=== 每個 domain 的 Accuracy ===
domain  count  correct accuracy
   CCD     39       26   66.67%
    中醫     60       28   46.67%

OVERALL Accuracy = 54.55%


RAGAs

In [ ]:
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path("key") / ".env"
load_dotenv(dotenv_path=env_path, override=False)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("找不到 OPENAI_API_KEY")

In [ ]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI  # langchain>=0.1
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)

In [ ]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness,ContextPrecision

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        LLMContextRecall(),
        ContextPrecision(),
        # Faithfulness(),      #only QA 忠實度
        # FactualCorrectness(), #only QA 正確性
    ],
    llm=evaluator_llm
)


In [ ]:
result

In [15]:
# ==== 建立輸出資料夾 ====
from pathlib import Path
OUT_DIR = Path("results")
OUT_DIR.mkdir(exist_ok=True)

# ==== 存檔 ====
csv_path = OUT_DIR / "baseline_results.csv"
test_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Baseline 結果已存到 {csv_path}")


✅ Baseline 結果已存到 results/baseline_results.csv
